In [ ]:
import operator
from typing import Annotated, TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage
from pydantic import BaseModel, Field
import os
from dotenv import load_dotenv

load_dotenv()

model = ChatGroq(model="llama-3.3-70b-versatile")

In [ ]:
class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(
        ..., description="Final evaluation verdict"
    )
    feedback: str = Field(
        ..., description="Constructive feedback for the tweet"
    )

structured_evaluator_llm = model.with_structured_output(TweetEvaluation)

In [ ]:
class TweetState(TypedDict):
    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iteration: int
    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

In [ ]:
def generate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".
Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english""")
    ]
    response = model.invoke(messages).content
    return {'tweet': response, 'tweet_history': [response]}

def evaluate_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic."),
        HumanMessage(content=f"""
Evaluate the following tweet: "{state['tweet']}"

Criteria:
1. Originality  - Is this fresh?
2. Humor        - Did it make you smile?
3. Punchiness   - Is it short and scroll-stopping?
4. Virality     - Would people retweet it?
5. Format       - Well-formed, under 280 chars?

Auto-reject if:
- Written in Q&A format
- Exceeds 280 characters
- Setup-punchline joke format

Respond ONLY as structured output.""")
    ]
    response = structured_evaluator_llm.invoke(messages)
    return {'evaluation': response.evaluation, 'feedback': response.feedback, 'feedback_history': [response.feedback]}

def optimize_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write as short, viral-worthy tweet. Avoid Q&A style. Under 280 chars.""")
    ]
    response  = model.invoke(messages).content
    return {'tweet': response, 'iteration': state['iteration'] + 1, 'tweet_history': [response]}

In [ ]:
def route_evaluation(state: TweetState):
    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    return 'needs_improvement'

In [ ]:
graph = StateGraph(TweetState)

graph.add_node('generate',  generate_tweet)
graph.add_node('evaluate',  evaluate_tweet)
graph.add_node('optimize',  optimize_tweet)

graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')
graph.add_conditional_edges(
    'evaluate',
    route_evaluation,
    {'approved': END, 'needs_improvement': 'optimize'}
)
graph.add_edge('optimize', 'evaluate')

workflow = graph.compile()

In [ ]:
initial_state = {
    'topic': 'AI replace human',
    'iteration': 0,
    'max_iteration': 3,
    'tweet_history': [],
    'feedback_history': []
}

result = workflow.invoke(initial_state)
import pprint
pprint.pprint(result)